In [ ]:
import pandas as pd
import numpy as np
import csv
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [ ]:
data = pd.read_csv('../assets/data/recommendation_dataset.csv', sep=';')

data.replace(99, np.nan, inplace=True)

data.iloc[:, 0] = data.iloc[:, 0].str.extract('(\d+)').astype(int)

data['DATASET'] = data['DATASET'].astype(int)

In [ ]:
data.dtypes

In [ ]:
# Iterar sobre cada columna (restaurante) para predecir los valores faltantes
for column in data.columns[0:]:  # Ignorar la primera columna de nombres de usuarios
    # Separar filas con datos conocidos y faltantes
    known_data = data[data[column].notna()]
    missing_data = data[data[column].isna()]

    # Continuar solo si hay datos faltantes
    if not missing_data.empty:
        # Definir características y etiquetas
        X_train = known_data.drop(columns=[column, 'DATASET'])
        y_train = known_data[column]
        X_missing = missing_data.drop(columns=[column, 'DATASET'])

        # Entrenar el modelo de XGBoost
        model = XGBRegressor(n_estimators=10, learning_rate=0.1, max_depth=1, random_state=42)
        model.fit(X_train, y_train)

        # Predecir los valores faltantes
        predictions = model.predict(X_missing)

        # Rellenar los valores predichos en el DataFrame
        data.loc[data[column].isna(), column] = predictions

# Guardar el DataFrame resultante
print("Datos imputados:")
print(data.head())
data.to_csv('../assets/data/dataset_XGBoost.csv', index=False, sep=',')

In [ ]:
def cargar_base_csv(ruta_base):
    """Carga las valoraciones desde el archivo base.csv, omitiendo la primera fila."""
    with open(ruta_base, 'r') as archivo_base:
        lector = csv.reader(archivo_base)
        next(lector, None)  # Omitir la primera fila
        return [list(map(float, fila)) for fila in lector]

def poblar_valoraciones(ruta_base, ruta_valoraciones, ruta_salida):
    """Llena el archivo valoraciones.csv con las valoraciones de base.csv."""
    # Cargar las valoraciones desde base.csv
    valoraciones_base = cargar_base_csv(ruta_base)

    # Leer el archivo valoraciones.csv y procesar las líneas
    with open(ruta_valoraciones, 'r') as archivo_val:
        lector = csv.reader(archivo_val, delimiter=';')
        filas_actualizadas = []
        
        for fila in lector:
            usuario = int(fila[0].replace('User', '')) - 1  # Índice del usuario
            restaurante = int(fila[1].replace('Restaurant', '')) - 1  # Índice del restaurante
            valoracion = valoraciones_base[usuario][restaurante]  # Obtener la valoración
            fila_actualizada = f"{fila[0]};{fila[1]};{valoracion}"
            filas_actualizadas.append(fila_actualizada)

    # Guardar las filas actualizadas en un archivo de salida
    with open(ruta_salida, 'w') as archivo_salida:
        for fila in filas_actualizadas:
            archivo_salida.write(fila + '\n')

# Ejecución de la función
ruta_base = '../assets/data/dataset_XGBoost.csv'
ruta_valoraciones = '../assets/data/valoraciones.csv'
ruta_salida = '../assets/data/dataset_result_XGBoost.csv'

poblar_valoraciones(ruta_base, ruta_valoraciones, ruta_salida)
print("Archivo dataset_result_XGBoost.csv generado exitosamente.")
print("python compute_mae.py target_recommendations.csv dataset_result_XGBoost.csv")

Archivo valoraciones_completadasXGBoost.csv generado exitosamente.
